# Talk2PowerSystem

## Setup logging

In [ ]:
import logging
import sys

logger = logging.getLogger('')
logger.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(logging.Formatter("%(asctime)s - %(levelname)s - %(message)s"))

logger.handlers.clear()
logger.addHandler(handler)

logging.getLogger("openai").setLevel(logging.ERROR)
logging.getLogger("httpx").setLevel(logging.ERROR)
logging.getLogger("httpcore").setLevel(logging.ERROR)

## Setup credentials

In [ ]:
from ttyg.utils import set_env

set_env("GRAPHDB_PASSWORD")
set_env("AZURE_OPENAI_API_KEY")

## Create the agent

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

from talk2powersystemllm import get_talk_to_power_system_agent

agent_executor = get_talk_to_power_system_agent("../../config/config.yaml", checkpointer=MemorySaver())

## Send consecutive questions in the same thread (conversation)

Note, that at the moment the conversations history is not persisted and is kept in the memory. Upon shut down of the notebook, it will be lost.

In [ ]:
from ttyg.agents import run_agent

conf = {"configurable": {"thread_id": "thread-123"}}
messages = {"messages": [("user", "List all transformers within substation OSLO.")]}
last_message_id = run_agent(agent_executor, messages, conf)

In [ ]:
messages = {"messages": [("user", "Give me their descriptions")]}
last_message_id = run_agent(agent_executor, messages, conf, last_message_id=last_message_id)

## Or iterate over set of questions, each within a separate thread (conversation)

In [ ]:
from ttyg.agents import run_agent

questions = [
    "List all transformers within substation OSLO.",
    "Liste alle transformatorer innenfor nettstasjon OSLO.",
    "List all substations within NO2 SGR region",
    "Liste alle nettstasjoner innenfor NO2 SGR regionen",
    "List all substations that are connected via an AC-line or a DC line to substation named ASKER",
    "List opp alle understasjoner som er koblet via en AC-linje eller en DC-linje til understasjon kalt ASKER",
    "List all AC-lines that traverse the regions NO5 SGR and NO2 SGR",
    "List opp alle AC-linjer som krysser regionene NO5 SGR og NO2 SGR",
    "show how many resources are there by class",
    "List five analogs of type active power",
    "Give me 8 synchronous machines of type generator",
    "List transformers that are normally in service",
    "Give me 5 switches that are normally closed",
    "List all synchronous machines that have \"M1\" or \"M2\" in the name, but not \"300\"",
    "Find the PSR f1769676",
    "Find the equipment f1769c74-9aeb-11e5-91da-b8763fd99c5f",
    "Give me the name of the bidding zone 10YNO-4--------9",
    "List all substations north of Trondheim",
]

for i, question in enumerate(questions):
    conf = {"configurable": {"thread_id": f"thread-{i}"}}
    messages = {"messages": [("user", question)]}
    run_agent(agent_executor, messages, conf)